# Graph Coloring Kata Workbook

**What is this workbook?**
A workbook is a collection of problems, accompanied by solutions to them. 
The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required. 

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Graph Coloring kata](./GraphColoring.ipynb). 
Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a first-time user.

## Part I. Colors Representation and Manipulation

### Task 1.1. Initialize register to a color

**Inputs:** 

  1. An integer $C$ ($0 \leq C \leq 2^{N} - 1$).

  2. An array of $N$ qubits in the $|0...0\rangle$ state.

**Goal:** 

Prepare the array in the basis state which represents the binary notation of $C$. 
Use little-endian encoding (i.e., the least significant bit should be stored in the first qubit).

**Example:** For $N = 2$ and $C = 2$ the state should be $|01\rangle$.

### Solution

We first need to convert the integer C to its binary representation. In Q#, we can use the [IntAsBoolArray](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.convert.intasboolarray) function to convert the input integer to its equivalent binary representation in little-endian `binaryC`.

Next we need to use `binaryC` as a bit mask: whenever `binaryC[i]` is 1 (or `true` if stored as an array of boolean values), we need to flip the qubit by applying an X gate. We can do this using a `for` loop:

In [ ]:
%kata T11_InitializeColor 

open Microsoft.Quantum.Convert;

operation InitializeColor (C : Int, register : Qubit[]) : Unit is Adj {
    let N = Length(register);
    // Convert C to an array of bits in little endian format
    let binaryC = IntAsBoolArray(C, N);
    // Value "true" corresponds to bit 1 and requires applying an X gate
    for i in 0 .. N - 1 {
        if (binaryC[i]) {
            X(register[i]);
        }
    }
}

Alternatively, we can use a helpful library operation [ApplyPauliFromBitString](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon.applypaulifrombitstring). 
It takes as input a Pauli operator $P \in \{I,X,Y,Z\}$, a boolean value, a boolean array and a qubit register and applies the Pauli operator to the register using the boolean array as a bit mask: the operator is applied to the qubits that correspond to array elements equal to the given boolean value.

We can think of `ApplyPauliFromBitString` as the following unitary transformation:

$$P^{b_0} \otimes P^{b_1} \otimes ... \otimes P^{b_{n-1}}$$

Here $P^0=I, P^1 =P$ (the given Pauli operator), and $b_i \in \{0,1\}$ are the elements of the given boolean array if the given boolean value is `true` or their negations if it is `false`. 

In our case, `ApplyPauliFromBitString(PauliX, true, binaryC, register)` represents the following transformation of `register`:

$$|\psi\rangle \xrightarrow{} X^{c_0} \otimes X^{c_1} \otimes ... \otimes X^{c_{n-1}}|\psi\rangle$$

where $c_0c_1...c_{n-1}$ is the binary representation of $C$.

When the input qubit register is in the state $|0...0\rangle$, `ApplyPauliFromBitString` operation will convert it into a basis state representing the boolean array, i.e., little-endian binary encoding of $C$:

$$|0...0\rangle \xrightarrow{} X^{c_0} \otimes X^{c_1} \otimes ... \otimes X^{c_{n-1}}|0...0\rangle = |c_0c_1...c_{n-1}\rangle = |C\rangle$$

In [ ]:
%kata T11_InitializeColor 

open Microsoft.Quantum.Convert;

operation InitializeColor (C : Int, register : Qubit[]) : Unit is Adj {
    let N = Length(register);
    // Convert C to an array of bits in little endian format
    let binaryC = IntAsBoolArray(C, N);
    // Value "true" corresponds to bit 1 and requires applying an X gate
    ApplyPauliFromBitString(PauliX, true, binaryC, register);
}

[Return to task 1.1 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-1.1.-Initialize-register-to-a-color)

### Task 1.2. Read color from a register

**Input:** An array of $N$ qubits which are guaranteed to be in one of the $2^{N}$ basis states.

**Output:** 

An $N$-bit integer that represents this basis state, in little-endian encoding. 
The operation should not change the state of the qubits.

**Example:** For $N = 2$ and the qubits in the state $|01\rangle$ return 2 (and keep the qubits in $|01\rangle$).

### Solution

Since we are guaranteed that `register` is in one of the $2^N$ basis states, 
simply measuring it (without resetting the qubits to the $|0\rangle$ state after the measurement) will not destroy any superposition and leave the state of the qubits unchanged, while giving us the necessary information. 

We can use the [MultiM](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.measurement.multim) operation to measure each of the qubits and store the result in an array of the type `Result[]`. 

We now need to convert these bits into an integer. We can do this directly using the [ResultArrayAsInt](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.convert.resultarrayasint) function which converts an array of `Result` values representing a little-endian encoding of an integer into the equivalent integer.

In [ ]:
%kata T12_MeasureColor 

open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Measurement;

operation MeasureColor (register : Qubit[]) : Int {
    let measurements = MultiM(register);
    return ResultArrayAsInt(measurements);
}

[Return to task 1.2 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-1.2.-Read-color-from-a-register)

### Task 1.3. Read coloring from a register

**Inputs:** 

  1. The number of elements in the coloring $K$.

  2. An array of $K * N$ qubits which are guaranteed to be in one of the $2^{KN}$ basis states.

**Output:** 

An array of $K$ $N$-bit integers that represent this basis state. 
$i$-th integer of the array is stored in qubits with indices $i * N$, $i * N + 1$, ..., $i * N + N - 1$ in little-endian format. 
The operation should not change the state of the qubits.

**Example:** 
For $N = 2$, $K = 2$ and the qubits in the state $|0110\rangle$ return `[2, 1]`.

### Solution

This can be considered as $K$-copies version of the previous task. In that task we read the color from a register of size $N$.
Here we have to return $K$ integers representing the $K$ colors from each of the $N$-bit registers. 

We are given $K$ and hence we can find out $N$ by dividing the `Length` of the register by $K$. 

Next we need to divide $KN$-qubit register into $K$ $N$-qubit registers. 
We use the Q# function [Chunks](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.arrays.chunks) which partitions the given array into chunks of the given length.

Finally, we use the [ForEach](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.arrays.foreach) operation to apply the `MeasureColor` operation from task 2.2 to each element of `colorPartitions` and assemble the results of each application into the resulting array `coloring`.

In [ ]:
%kata T13_MeasureColoring 

open Microsoft.Quantum.Arrays;

operation MeasureColoring (K : Int, register : Qubit[]) : Int[] {
    let N = Length(register) / K;
    let colorPartitions = Chunks(N, register);
    let coloring = ForEach(MeasureColor, colorPartitions);
    return coloring;
}

[Return to task 1.3 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-1.3.--Read-coloring-from-a-register)

### Task 1.4. 2-bit color equality oracle

**Inputs:** 

  1. An array of 2 qubits in an arbitrary state $|c_{0}\rangle$ representing the first color.

  2. An array of 2 qubits in an arbitrary state $|c_{1}\rangle$ representing the second color.

  3. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|c_{0}\rangle|c_{1}\rangle|y\rangle$ into state $|c_{0}\rangle|c_{1}\rangle|y \oplus f(c_{0},c_{1})\rangle$ ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if $c_{0}$ and $c_{1}$ are in the same state, and 0 otherwise. 
Leave the query register in the same state it started in.

In this task you are allowed to allocate extra qubits.

### Solution

We are given that $f(c_0,c_1)=1$ if and only if $c_0=c_1$. Let's express this using XOR (or $\oplus$) - a binary operation defined as follows:

$$c_0 = c_1 \Leftrightarrow c_0 \oplus c_1 =0 $$

We can express $f(c_0,c_1)$ as $1$ if $c_0\oplus c_1=0$ and $0$ otherwise. The advantage of this representation over the previous one is that we can calculate the XOR of two bits using the $\textrm{CNOT}$ operator. 
To do this, we allocate an extra qubit in the $|0\rangle$ state and do two $\textrm{CNOT}$s with each of the input bits as the control and the extra qubit as target. Since the effect of $\textrm{CNOT}$ is $|x\rangle|y\rangle \rightarrow |x\rangle|y \oplus x\rangle$, the effect of such a pair of $\textrm{CNOT}$s will be 

$$|b_0b_1\rangle|0\rangle \rightarrow |b_0b_1\rangle|(0 \oplus b_0) \oplus b_1\rangle = |b_0b_1\rangle|b_0 \oplus b_1\rangle$$

Thus, we can compute bitwise XOR of bit strings $c_0$ and $c_1$ by allocating two auxiliary qubits $|a\rangle$ in the initial state $|00\rangle$ and applying the XOR computation procedure described above to pairs of corresponding bits in $c_0$ and $c_1$.

We now need to flip the target qubit $|y\rangle$ only if the auxiliary qubits $|a\rangle$ are in the $|00\rangle$ state. 
This can be done by using zero-controlled $X$ gate, i.e., `ControlledOnInt(0, X)`.
Finally, we need to uncompute the bitwise XOR to ensure the auxiliary qubits are again in the $|00\rangle$ state before releasing them.

In [ ]:
%kata T14_ColorEqualityOracle_2bit 

operation ColorEqualityOracle_2bit (c0 : Qubit[], c1 : Qubit[], target : Qubit) : Unit is Adj+Ctl {
    use a = Qubit[2];
    within {
        // Compute bitwise XOR of c0 and c1 and store it in a
        CNOT(c0[0],a[0]);
        CNOT(c0[1],a[1]);
        CNOT(c1[0],a[0]);
        CNOT(c1[1],a[1]);
    } apply {
        // If all XORs are 0, c0 = c1, and our function is 1
        (ControlledOnInt(0, X))(a, target);
    }
}

[Return to task 1.4 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-1.4.-2-bit-color-equality-oracle)

### Task 1.5. N-bit color equality oracle (no extra qubits)

This task is the same as task 1.4, but in this task you are NOT allowed to allocate extra qubits.

### Solution

Since this task is the generalized $N$-bit version of the previous task, we can approach it in a similar manner: compute the bitwise XOR of $N$-bit registers $c_0$ and $c_1$, and flip the target qubit if the XOR is $0$. 
However, this time we are not allowed to allocate extra qubits and thus must compute (and uncompute) XOR in-place.

We can do this by storing $c_1 \oplus c_0$ in $c_1$ itself: this is exactly the effect of the $\textrm{CNOT}$ gate!
We'll use $N$ $\textrm{CNOT}$ gates, with each of the qubits of $c_0$ acting as the control and the respective qubits of $c_1$ acting as the target.
The remaining procedure is exactly the same as the solution to task 1.4.

In [ ]:
%kata T15_ColorEqualityOracle_Nbit 

operation ColorEqualityOracle_Nbit (c0 : Qubit[], c1 : Qubit[], target : Qubit) : Unit is Adj+Ctl {
    within {
        // Compute bitwise XOR of c0 and c1 in place (storing it in c1)
        for i in 0 .. Length(c0) - 1 {
            CNOT(c0[i], c1[i]);
        }
    } apply {
        // If all XORs are 0, c0 = c1, and our function is 1
        (ControlledOnInt(0, X))(c1, target);
    }
}

[Return to task 1.5 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-1.5.-N-bit-color-equality-oracle-(no-extra-qubits))

## Part II. Vertex coloring problem

### Task 2.1. Classical verification of vertex coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $V$ integers, representing the vertex coloring of the graph. 
$i$-th element of the array is the color of the vertex number $i$.

**Output:** 

True if the given vertex coloring is valid (i.e., no pair of vertices connected by an edge have the same color), and false otherwise.

**Example:** 

Graph 0 -- 1 -- 2 would have $V = 3$ and `edges = [(0, 1), (1, 2)]`.  
Some of the valid colorings for it would be `[0, 1, 0]` and `[-1, 5, 18]`.

### Solution

A graph coloring valid when the nodes connected by every edge have a different color. This means that we have to check every edge, see if the nodes have the same color, and if it is the case, return that the graph coloring is invalid. If every edge passed the test, we can safely say the graph coloring is valid.

Since the color of vertex $n$  is the $n$-th element of the `colors` array, we simply loop through every edge, which is a pair of vertices, and compare corresponding colors.

In [ ]:
%kata T21_IsVertexColoringValid 

function IsVertexColoringValid (V : Int, edges: (Int, Int)[], colors: Int[]) : Bool {
    // Loop through every edge
    for (v0, v1) in edges {
        // Compare the colors of vertices
        if (colors[v0] == colors[v1]){
            // A return statement stops the execution of the function
            return false;
        }
    }
    // If the code reaches this point, every edge was correct
    return true;
}

[Return to task 2.1 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-2.1.-Classical-verification-of-vertex-coloring)

### Task 2.2. Oracle for verifying vertex coloring

**Inputs:**

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph (E $\leq$ 12).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $2V$ qubits `colorsRegister` that encodes the color assignments.

  4. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$  ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if the given vertex coloring is valid, and 0 otherwise. 
Leave the query register in the same state it started in.

Each color in `colorsRegister` is represented as a 2-bit integer in little-endian format. 
See task 1.3 for a more detailed description of color assignments.

### Solution

The oracle will rely on the `ColorEqualityOracle_Nbit` operation from task 1.5. 
We will allocate an array of qubits, of the same size as the number of edges, that will be flipped if the vertices connected by the corresponding edge have the same color. 
This can easily be done with the operation defined in task 1.5. We will then check if the array is still in state $|0...0\rangle$; if it is, the coloring is valid.

Since the coloring is provided as an array of qubits, with 2 qubits per vertex (2 qubits = 4 basis states = 4 colors), we have to take the correct chunks of the coloring. We can deduce that the coloring of vertex $n$ is encoded in qubits in positions $2*n$ and $2*n+1$.

Also, do not forget to uncompute to leave the qubits clean.

> In Q#, a sub-array of array elements between indices $a$ and $b$, inclusive, is written as `array[a..b]` (see [array slicing documentation](https://docs.microsoft.com/azure/quantum/user-guide/language/expressions/itemaccessexpressions)).
>
> The uncomputing of the temporarily allocated qubits can be done using the `within ... apply ...` structure (see [conjugations documentation](https://docs.microsoft.com/azure/quantum/user-guide/language/statements/conjugations)).

In [ ]:
%kata T22_VertexColoringOracle 

operation VertexColoringOracle (V : Int, 
                                edges : (Int, Int)[], 
                                colorsRegister : Qubit[], 
                                target : Qubit) : Unit is Adj+Ctl {
    // Store the number of edges
    let edgesNumber = Length(edges);
    // Allocate the array of qubits storing the conflicts of the coloring 
    use conflicts = Qubit[edgesNumber];
    within {
        // Iterate over every edge
        for i in 0 .. edgesNumber - 1 {
            // Deconstruct the edge tuple into two separate vertex indices
            let (v0, v1) = edges[i];
            // Use the operation from task 1.5 to track conflicts
            ColorEqualityOracle_Nbit(colorsRegister[2*v0 .. 2*v0+1], 
                                     colorsRegister[2*v1 .. 2*v1+1], conflicts[i]);
        }
    } apply {
        // If all the edges are colored properly, conflicts should be in state |0...0⟩
        (ControlledOnInt(0, X))(conflicts, target);
    }
}

[Return to task 2.2 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-2.2.-Oracle-for-verifying-vertex-coloring)

### Task 2.3. Using Grover's search to find vertex coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. A marking oracle which implements vertex coloring verification, as implemented in task 2.2.

**Output:** 

A valid vertex coloring for the graph, in a format used in task 2.1.

### Solution

First we have to implement the generic Grover's algorithm. However, learning the implementation details is not the goal of this kata, so we will assume that you are already familiar with it; if not, please refer to the [Grover's Algorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb). 

The first code cell implements the phase kickback trick to turn a marking oracle into a phase oracle. The second one is the actual implementation of Grover's algorithm.

In [ ]:
operation OracleConverter (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit is Adj {
    use target = Qubit();
    within {
        // Put the target qubit in the |-⟩ state
        X(target);
        H(target);
    } apply {
        // Apply the marking oracle
        markingOracle(register, target);
    }
}

In [ ]:
open Microsoft.Quantum.Arrays;

operation GroverAlgorithmLoop (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[], iterations : Int) : Unit is Adj {
    // Convert the marking oracle in a phase oracle
    let phaseOracle = OracleConverter(markingOracle, _);
    // Prepare an equal superposition of all basis states
    ApplyToEachA(H, register);
    // Apply Grover iterations
    for _ in 1..iterations {
        // Apply phase oracle
        phaseOracle(register);
        // Apply "reflection about the mean"
        within {
            ApplyToEachA(H, register);
            ApplyToEachA(X, register);
        } apply {
            (Controlled Z)(Most(register), Tail(register));
        }
    }
}

The main part of this task is running Grover's algorithm to find a solution. 

To do this, we'll need to take some extra steps to use the generic implementation. Here is the flow:
- Allocate an array of qubits to store graph coloring with 2 qubits per vertex and one more qubit to use when we verify that the solution we found is indeed correct.
- Try running the algorithm with different numbers of Grover's iterations, starting with 1 iteration and increasing the number each time we don't find a solution. 
We will use two mutable variables for this, `iterations` to store iteration count and `correct` to indicate whether we found a correct solution, and the [repeat-until-success loop](https://docs.microsoft.com/azure/quantum/user-guide/language/statements/conditionalloops#repeat-statement).
- In the body of the loop we'll do the following steps:
    - Use Grover's algorithm loop implemented in a previous code cell with the current number of iterations.
    - Measure the qubit array to the result of Grover's algorithm. We can do that using the [`MultiM` operation](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.measurement.multim).
    - Check whether the result is a valid graph coloring by applying the marking oracle to the qubit array that stores the coloring (remember that after the measurement the state of these qubits collapsed to the basis state that corresponds to the measurement results) and the extra qubit. 
    - Measure the extra qubit in the Pauli Z basis and reset it using the [`MResetZ` operation](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.measurement.mresetz):
        - If the measurement result is `One`, the algorithm result is indeed a solution to the problem we're solving; we need to set `correct` variable to `true` and to decode the result into a graph coloring using the `MeasureColoring` operation from task 1.3.
        - If the measurement result is `Zero`, the algorithm result is not a solution to our problem, and we do nothing.
    - Reset the array that stored the coloring to prepare it for the next iteration using the [`ResetAll` operation](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.resetall).
- We stop the loop if we found the solution or if we are running too many iterations (in this case we throw an exception to indicate that we didn't find a solution).

In [ ]:
%kata T23_GroversAlgorithm 

open Microsoft.Quantum.Measurement;

operation GroversAlgorithm (V : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Int[] {
    mutable coloring = new Int[V];
    use (register, output) = (Qubit[2 * V], Qubit());
    mutable correct = false;
    mutable iterations = 1;
    repeat {
        Message($"Trying iteration {iterations}");
        GroverAlgorithm(oracle, register, iterations);
        let temp = MultiM(register);
        oracle(register, output);
        if (MResetZ(output) == One) {
            set correct = true;
            set coloring = MeasureColoring(V, register);
        }
        ResetAll(register);
    }
    until (correct or iterations > 10)
    fixup {
        set iterations += 1;
    }
    if (not correct) {
        fail "No valid coloring was found";
    }
    return coloring;
}

[Return to task 2.3 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-2.3.-Using-Grover's-search-to-find-vertex-coloring)